# P. Visión

In [2]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure

In [3]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

In [4]:
path = 'C:\\Users\\alumno\\Desktop\\train_pos'
fotosPosTrain = leerFotos(path)

path = 'C:\\Users\\alumno\\Desktop\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

In [5]:
def generaImagenesRecortadas(dataPos,dataNeg):    
    numeroAleatoriasPorFoto = 3       # Numero de recortadas aleatorias por foto
    rowsFotoPersona = dataPos[0].shape[0]
    colsFotoPersona = dataPos[0].shape[1]

    rowsFotoEntorno = dataNeg[0].shape[0]
    colsFotoEntorno = dataNeg[0].shape[1]

    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        for j in range(numeroAleatoriasPorFoto):
            randRow = random.randint(0,rowsFotoEntorno - rowsFotoPersona)
            randCol = random.randint(0,colsFotoEntorno - colsFotoPersona)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroAleatoriasPorFoto*i] = randomImage 
    return fotosNegTrain

In [6]:
fotosNegTrain = generaImagenesRecortadas(fotosPosTrain,fotosNegTrainSinTratar)

In [ ]:
fdPosTrain = np.zeros((len(fotosPosTrain),6804))
for i in range(len(fotosPosTrain)):
#fd,imHog = hog(fotosPosTrain[1], orientations=8, pixels_per_cell=(8, 8),cells_per_block=(3, 3),visualize=True)
    fdPosTrain[i] = hog(fotosPosTrain[i])
    #imHog_re = exposure.rescale_intensity(imHog, in_range = (0,10))
print(fdPosTrain.shape,fotosPosTrain[1].shape,134*70)

In [7]:
def vectolizarHOG(imagenes,esPositivo):
    ### Esata funcion aplica HOG a todas las imagenes del diccionario
    
    #tamaño del vector
    tamVector = len(hog(imagenes[0]))
    
    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenes = np.zeros((len(imagenes),tamVector))
    
    for i in range(len(imagenes)):
        fdImagenes[i] = hog(imagenes[i])
    
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenes,y

In [8]:
fdPosTrain,yPos = vectolizarHOG(fotosPosTrain,1)

fdNegTrain,yNeg = vectolizarHOG(fotosNegTrain,0)

print(fdPosTrain.shape,yPos.shape)
print(fdNegTrain.shape,yNeg.shape)

(563, 6804) (563, 1)
(600, 6804) (600, 1)


In [9]:
fdTrain = np.vstack((fdPosTrain,fdNegTrain))
y = np.vstack((yPos,yNeg))

print(fdTrain.shape,y.shape)

(1163, 6804) (1163, 1)


In [10]:
from sklearn import svm

clf = svm.SVC()
clf.fit(fdTrain,y.ravel())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [11]:
pred = clf.predict(fdTrain[0].reshape(1, -1))
print(clf.score(fdTrain,y))

0.9312123817712812


In [ ]:
# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(fotosPosTrain[2],(3,3),0)
th3 = cv2.Canny(blur,50,200)

In [ ]:
cv2.imshow("sadas",blur)
cv2.imshow("ds2da",th3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
print()